In [ ]:
################## auto login ######################
import login
from login import *
import pandas as pd
import pytz
from time import sleep
import datetime

In [ ]:
############################### Parameters ####################################

symbol_ip = 'USDINR20DECFUT' # USDINR20DECFUT / GBPINR20OCTFUT / EURINR20OCTFUT / JPYINR20OCTFUT
inst_token = 380675          # 380675(USD) / 490755(GBP) / 278019(EUR) / 690435(JPY)
order_type = 'MIS'           # MIS / NRML
one_dollar_margin = 954     # (USD-MIS=984/NRML=1967)(GBP-MIS=1948/NRML=3895)(EUR-MIS=1313/NRML=2625)(JPY-MIS=1333/NRML=2667)
exchange_type=kite.EXCHANGE_CDS
fullquantity = 125           # Set a required quantity or set 0 (zero) to adjust quantity automatically
offset_qty = 0               # !!!!!!!!!!!!!!without point!!!!!!!!!!!!!!!!!!!
powermulti = 2               # Set a multiplyer number to multiply quantity at every part (level)

stoploss_on = False          # True / False
auto_convert = False
days_ago = 2

In [ ]:
# prev_close = kite.quote([inst_token])[str(inst_token)]['ohlc']['close']
# prev_close

In [ ]:
import datetime
interval='day'
todaydt=datetime.date.today()
hud_ago=todaydt-datetime.timedelta(days=10)
to_date=datetime.date.isoformat(todaydt)

from_date=datetime.date.isoformat(hud_ago)
data = kite.historical_data(inst_token, from_date, to_date, interval, continuous=False)

import pandas as pd
mydata = pd.DataFrame(data)
from datetime import date
today = date.today()

print('Today Date : '+str(today))
yest_date =  mydata[-days_ago:][['date']].to_numpy()[0][0]
last_close =  mydata[-days_ago:][['close']].to_numpy()[0][0]
last_high =  mydata[-days_ago:][['high']].to_numpy()[0][0]
last_low =  mydata[-days_ago:][['low']].to_numpy()[0][0]
hl_range = round(last_high - last_low,4)

pivot_range = hl_range
print('Yestreday Date : '+str(yest_date))       
print('last close : '+ str(last_close))
print('last high : '+ str(last_high))
print('last low : '+ str(last_low))
print('HL Range : '+ str(round(pivot_range,4)))

mymargin = round(kite.margins(segment = 'equity')['available']['live_balance'],2)

pivot_price = last_close

if (fullquantity==0):
    fullquantity = int(mymargin / one_dollar_margin)

print('My Margin : '+str(mymargin))
print('Pivot Price : ' + str(pivot_price))
print('Full Quantity : '+str(fullquantity))
print('Stoploss On ? : '+str(stoploss_on))
print('Order Type : '+str(order_type))
print()

total_parts = 14

scounter = int(total_parts/2)

fibonmulti1=0.1180
fibonmulti2=0.2360
fibonmulti3=0.3820
fibonmulti4=0.6180
fibonmulti5=1.0000  # 100% Range
fibonmulti6=1.1180
fibonmulti7=1.2360
fibonmulti8=1.3820
fibonmulti9=1.6180
fibonmulti10=2.0000 # Maximum quantity @ 200% Range
fibonmulti11=2.1180 # Stops trading here
fibonmulti12=2.3820 # Stoploss if set 'True' / No Stoploss if set 'False'

print()
print('pivot price : ' +str(pivot_price))

print()
base=0.0025

for i in range(1, int(total_parts/2)+1):
    globals()['p'+str(i)+'b'] = round(pivot_price - pivot_range*(globals()['fibonmulti'+str(i)]),4)
    globals()['p'+str(i)+'b'] = round(base * round(globals()['p'+str(i)+'b'] /base),4)
    print('p'+str(i)+'b = '+str(globals()['p'+str(i)+'b']))

print()
for i in range(1, int(total_parts/2)+1):
    globals()['p'+str(i)+'s'] = round(pivot_price + pivot_range*(globals()['fibonmulti'+str(i)]),4)
    globals()['p'+str(i)+'s'] = round(base * round(globals()['p'+str(i)+'s'] /base),4)
    print('p'+str(i)+'s = '+str(globals()['p'+str(i)+'s']))

print()

halfqunat = fullquantity/1
tempcounter = 1

if(powermulti==1):
    for i in range(4,-1,-1):
        globals()['p'+str(tempcounter)+'q'] =int(fullquantity-(i*fullquantity/5))
        print('p'+str(tempcounter)+'q : ' + str(globals()['p'+str(tempcounter)+'q']))
        tempcounter = tempcounter+1
else:
    for j in range(1):
        for i in range(4,-1,-1):
            globals()['p'+str(tempcounter)+'q'] = int((halfqunat*j) + int(halfqunat/pow(powermulti,i)))
            print('p'+str(tempcounter)+'q : ' + str(globals()['p'+str(tempcounter)+'q']))
            tempcounter = tempcounter+1

p6q = p5q
if(stoploss_on==True):
    p7q = 0
else :
    p7q = p6q

print('p6q : '+ str(p6q))
print('p7q : '+ str(p7q))

const_param_map = {'Today Date': str(today),
                 'Yesterday Date':str(yest_date),
                 'Last High': last_high,
                 'Last Low': last_low,
                 'Last Close':last_close,
                 'HL Range': hl_range,
                 'My Margin': mymargin
                  }

var_param_map = {'Pivot Price':pivot_price,
                 'Full Quantity':fullquantity,
                 'Stoploss on?':stoploss_on,
                 'Order Type': order_type}

pivots_map = {'Buy':[p1b,p2b,p3b,p4b,p5b,p6b,p7b],
              'Quantity':[p1q,p2q,p3q,p4q,p5q,p6q,p7q],
              'Sell':[p1s,p2s,p3s,p4s,p5s,p6s,p7s]}

const_param_map_df = pd.DataFrame(const_param_map, index=[0])
var_param_map_df = pd.DataFrame(var_param_map, index=[0])
pivots_map_df = pd.DataFrame(pivots_map, index=[0,1,2,3,4,5,6])


const_param_map_df.to_csv('./const_param_map.csv')
var_param_map_df.to_csv('./var_param_map.csv')
pivots_map_df.to_csv('./pivots_map.csv')


In [ ]:
####################### Function Definition ############################
myquantity = 0
def getquant():
    global myquantity, order_type, symbol_ip
    allpos = kite.positions()['net']
    for i in range(len(allpos)):
        if(allpos[i]['tradingsymbol']==symbol_ip and  allpos[i]['product']== order_type):
            myquantity = allpos[i]['quantity']
            print('My Quantity : ' + str(allpos[i]['quantity']))
        
getquant()

stop_trading = False
def placeneworder(quantdiff):
    global symbol_ip, stop_trading,exchange_type
    if(quantdiff>0 and stop_trading==False):
        try:
            stop_trading = True
            order_id= kite.place_order(tradingsymbol=symbol_ip,
                        exchange=exchange_type,
                        transaction_type=kite.TRANSACTION_TYPE_BUY,
                        quantity=abs(quantdiff)-offset_qty,
                        order_type=kite.ORDER_TYPE_MARKET,
                        variety = kite.VARIETY_REGULAR,
                        product=kite.PRODUCT_MIS if order_type=='MIS' else kite.PRODUCT_NRML)
            getquant()
            stop_trading = False
        except Exception as e:
            stop_trading = True
            print(e)
            getquant()
            stop_trading = False
    if(quantdiff<0 and stop_trading==False):
        try:
            stop_trading = True
            order_id= kite.place_order(tradingsymbol=symbol_ip,
                        exchange=exchange_type,
                        transaction_type=kite.TRANSACTION_TYPE_SELL,
                        quantity=abs(quantdiff)-offset_qty,
                        order_type=kite.ORDER_TYPE_MARKET,
                        variety = kite.VARIETY_REGULAR,
                        product=kite.PRODUCT_MIS if order_type=='MIS' else kite.PRODUCT_NRML)
            getquant()
            stop_trading = False
        except Exception as e:
            stop_trading = True
            print(e)
            getquant()
            stop_trading = False


def ckqnt (orderprice,orderquant):
    global myquantity
    quantdiff = orderquant - myquantity
    placeneworder(quantdiff)
    return myquantity


def checkpivot(ltps):
    global myquantity
    if (ltps==pivot_price):
        ckqnt(pivot_price,0)
        print('in pivot range')
    this_point = 0
    for i in range(1,int(total_parts/2)+1):
        if(i==int(total_parts/2)):
            if (ltps <= (globals()['p'+str(i)+'b']) and stoploss_on==True ):
                ckqnt(globals()['p'+str(i)+'b'],globals()['p'+str(i)+'q'])
                print('==========' + 'p'+str(i)+'b' +'===========')
                this_point = i
                print(str(ltps)+ ' < ' + 'p'+str(i)+'b')
            if (ltps >= (globals()['p'+str(i)+'s']) and stoploss_on==True):
                ckqnt(globals()['p'+str(i)+'s'],-(globals()['p'+str(i)+'q']))
                print('==========' + 'p'+str(i)+'b' +'===========')
                this_point = i
                print(str(ltps)+ ' < ' + 'p'+str(i)+'b')
        
        else:
            if (ltps == globals()['p'+str(i)+'b']):
                ckqnt(globals()['p'+str(i)+'b'],globals()['p'+str(i)+'q'])
                this_point = i
                print('==========' +'p'+str(i)+'b' + '==============' )
                print(str(globals()['p'+str(i+1)+'b'])+ ' < ' +str(ltps)+ ' < ' +str(globals()['p'+str(i)+'b']))
            if (ltps == globals()['p'+str(i)+'s']):
                ckqnt(globals()['p'+str(i)+'s'],-(globals()['p'+str(i)+'q']))
                this_point = i
                print('==========' +'p'+str(i)+'s' + '==============' )
                print(str(globals()['p'+str(i)+'s'])+ ' < ' +str(ltps)+ ' < ' +str(globals()['p'+str(i+1)+'s']))
                
            if(ltps <= globals()['p'+str(i)+'b'] and ltps > globals()['p'+str(i+1)+'b']):
                this_point=i   
            if(ltps >= globals()['p'+str(i)+'s'] and ltps < globals()['p'+str(i+1)+'s']):
                this_point=i

    if(myquantity>0):
        
        
        map1 = {'Previous':[globals()['p'+str(this_point)+'b'],
                            globals()['p'+str(this_point)+'q']],
                'Current':[ltps,myquantity],
                'Next':[globals()['p'+str(this_point+1)+'b'],
                        globals()['p'+str(this_point+1)+'q']]}
        
        mydf1 = pd.DataFrame(map1, index=['Price','Quantity'])
        mydf1.to_csv('./mycsv.csv')
        
    if(myquantity<0):
        map1 = {'Previous':[globals()['p'+str(this_point)+'b'],
                            globals()['p'+str(this_point)+'q']],
                'Current':[ltps,myquantity],
                'Next':[globals()['p'+str(this_point+1)+'s'],
                        -globals()['p'+str(this_point+1)+'q']]}
        
        mydf1 = pd.DataFrame(map1, index=['Price','Quantity'])
        mydf1.to_csv('./mycsv.csv')
        
    if(myquantity==0):
        map1 = {'Previous':[p1b, p1q],
                'Current':[ltps,myquantity],
                'Next':[p1s, -p1q],}
        
        mydf1 = pd.DataFrame(map1, index=['Price','Quantity'])
        
        mydf1.to_csv('./mycsv.csv')


In [ ]:
# ############### Convert Position @ 16:40 #####################
# import datetime
# from time import sleep

# def act(x):
#     kite.convert_position(
#                          exchange = exchange_type,
#                          tradingsymbol = symbol_ip,
#                          transaction_type = kite.TRANSACTION_TYPE_SELL if myquantity <0 else kite.TRANSACTION_TYPE_BUY ,
#                          position_type='day',
#                          quantity=myquantity/2,
#                          old_product = kite.PRODUCT_MIS,
#                          new_product = kite.PRODUCT_NRML)
#     stop_trading = True
    
    
#     return x+10

# def wait_start(runTime, action):
#     startTime = datetime.time(*(map(int, runTime.split(':'))))
#     while startTime > datetime.datetime.today().time(): 
#         sleep(1)
#     return action

# wait_start('13:43', lambda: act(100))

In [ ]:
# kite.positions()

In [ ]:
# wait_start('13:43',)

In [ ]:
################## START TRADING #######################
moving_pivot_on = 2
import logging
import threading
from kiteconnect import KiteTicker
logging.basicConfig(level=logging.DEBUG)
# kws = KiteTicker("w19o0chuo929jxkp", "eA5B5OJQNOtZ0OYihkBmYw7Ke3B9pmCC")

this_second = datetime.datetime.now().second
last_second = 0

def on_ticks(ws, ticks):
    global pivot_price, moving_pivot_on, this_second, last_second, stop_trading
    this_second = datetime.datetime.now().second
    if(last_second!=this_second):
        last_second = this_second
        x = threading.Thread(target=checkpivot, args=([ticks[0]['last_price']]))
        if(stop_trading==False):
            x.start()
            
#     checkpivot(ticks[0]['last_price'])

#     if(moving_pivot_on):
#         pivot_price =round(base * round(ticks[0]['average_price']/base),4) # Moving Trade Average Line : 1
#         calc_pivots();                                                     # Moving Trade Average Line : 2
        
    #print('===========================')
    #print(type(ticks[0]['last_price']))
    print('==========================')
    mymap = {'ltp':ticks[0]['last_price']}
    mynewdf = pd.DataFrame(mymap, index=[0])
    mynewdf.to_csv('./ltp.csv')
    
    print( 'LTP : ' + str(ticks[0]['last_price']))
    #print('==========================')
    #print()

def on_connect(ws, response):
    global inst_token
    ws.subscribe([inst_token])
    ws.set_mode(ws.MODE_FULL, [inst_token])

def on_close(ws, code, reason):
    ws.stop()
    
def on_error(ws, code, reason):
    logging.error("closed connection on error: {} {}".format(code, reason))

def on_noreconnect(ws):
    logging.error("Reconnecting the websocket failed")

def on_reconnect(ws, attempt_count):
    logging.debug("Reconnecting the websocket: {}".format(attempt_count))

#def on_order_update(ws, data):
#    print("order update: ", data)

kws.on_error = on_error
kws.on_noreconnect = on_noreconnect
kws.on_reconnect = on_reconnect
#kws.on_order_update = on_order_update

kws.on_ticks = on_ticks
kws.on_connect = on_connect
kws.on_close = on_close

kws.connect()